In [53]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import seaborn as sns
import statsmodels.api as sm


In [54]:
def load_peace_sys_data():
    df = pd.DataFrame(pd.read_csv('peace_sys.csv', index_col=0, na_values=['(NA)']))
    return df 

In [55]:
def get_odds_ratios(coefs):
    return np.exp(coefs[0])

In [56]:
def logit_pvalue(model, x):
    """ Calculate z-scores for scikit-learn LogisticRegression.
    Parameters
    ---------
        model: fitted sklearn.linear_model.LogisticRegression with intercept and large C
        x:     matrix on which the model was fit
    This function uses asymtptics for maximum likelihood estimates.
    """
    # first index refers to proba that belongs to class 0
    # second index refers to proba that belongs to calss 1
    p = model.predict_proba(x) # return matrix (N,2)
    # number of samples
    n = len(p)
    # number of features + 1 
    m = len(model.coef_[0]) + 1
    coefs = np.concatenate([model.intercept_, model.coef_[0]]) # put intercept and coefs in same array
    x_full = np.matrix(np.insert(np.array(x), 0, 1, axis = 1)) # 
    ans = np.zeros((m, m))
    for i in range(n):
        # dot product of transposed row and row
        # then multiply by both probas
        # add it to ans
        ans = ans + np.dot(np.transpose(x_full[i, :]), x_full[i, :]) * p[i,1] * p[i, 0]
    # acovariance matrix
    vcov = np.linalg.inv(np.matrix(ans))
    # square root diagonal of covariace matrix
    se = np.sqrt(np.diag(vcov))
    # divide coefs by standard error
    t =  coefs/se 
    # two tailed using normal dist
    p = (1 - stats.norm.cdf(abs(t))) * 2
    return p

In [57]:
def logit_with_nan(data):
    data_copy = data.drop('PSys', axis=1)
    pvalues = []
    logit_beta = []
    logit_odds_ratio = []

    
    for label in data_copy.columns:
        X = data[[label, 'PSys']]
        X = X.dropna()
        
        y = X.PSys
        X = X.drop('PSys', axis=1)
        
        logreg = LogisticRegression(random_state=42, C=1e9)
        logreg.fit(X, y)
        pvalue = logit_pvalue(logreg, X)[1]
        
        pvalues.append(pvalue)
        
        beta = logreg.coef_[0][0]
        logit_beta.append(beta)
        logit_odds_ratio.append(np.exp(beta))
        
        
    res = pd.DataFrame({'Variable' : data_copy.columns,
                        'Logistic_Sig_Level' : pvalues,
                        'Logistic_Beta' : logit_beta,
                        'Logistic_Odds_Ratio' : logit_odds_ratio
                       })
    
    res = res.set_index('Variable')
    
    return res

In [58]:
def compute_ttest(peaceful, non_peaceful):
    variables = peaceful.columns
    res_df = pd.DataFrame(columns=['feature', 't_statistic', 'p_value'])
    
    for var in variables:
        x = peaceful[var]
        y = non_peaceful[var]
        ttest = stats.ttest_ind(x, y)
        temp = pd.DataFrame({'feature' : [var] , 
                             't_statistic' : ttest.statistic, 
                             'p_value' : [ttest.pvalue]})
        res_df = res_df.append(temp)
        
    res_df.set_index('feature', inplace=True)
    return res_df

In [59]:
def ttest_with_nan(data):
    """ 
    This is a two sided test for the null hypothesis that 2 independent samples
    have identical average(expected) values.
    
    Test assusmes that the populations have identical variances by default
    
    Parameters
    ----------
    data: DataFrame
        The DataFrame that contains Nan values
        
    Returns
    -------
    returns DataFrame that contains statistic and pvalue for every column in data
    
    t_statistic: float
        Calculated t-statistic
    
    p_value: float
        Two tailed p-value
            
    """
    # create a copy that excludes target variable
    data_copy = data.drop('PSys', axis=1)
    # prepare result data frame
    res_df = pd.DataFrame(columns=['Variable', 't_statistic', 'p_value'])
    # for every label, we separate each row by what its target value is
    # we drop the nan values
    for label in data_copy.columns:
        X = data[[label, 'PSys']]
        X = X.dropna()
        # filter peaceful 
        peaceful = X[X.PSys == 1]
        peaceful = peaceful.drop('PSys', axis=1)
        
        non_peaceful = X[X.PSys == 2]
        non_peaceful = non_peaceful.drop('PSys', axis=1) 
        # calculate two-sided ttest
        ttest_result = stats.ttest_ind(peaceful, non_peaceful)
        ttest_df = pd.DataFrame({'Variable' : [label],
                                 't_statistic' : ttest_result.statistic,
                                 'p_value' : ttest_result.pvalue
                                })
        
        res_df = res_df.append(ttest_df)
    
    res_df.set_index('Variable', inplace=True)
    
    return res_df

In [60]:
def compute_mannwhitneyu(peaceful, non_peaceful):
    variables = peaceful.columns
    res_df = pd.DataFrame(columns=['feature', 'statistic', 'p_value'])
    
    for var in variables:
        x = peaceful[var]
        y = non_peaceful[var]
        utest = stats.mannwhitneyu(x, y)
        temp = pd.DataFrame({'feature' : [var] , 
                             'statistic' : utest.statistic, 
                             'p_value' : [utest.pvalue]})
        res_df = res_df.append(temp)
        
    res_df.set_index('feature', inplace=True)
    return res_df

In [61]:
def mannwhitneyu_with_nan(data):
    """ 
    Computes the Mann-Whitney rank test on each of the columns of data
    Each column is seperated into 2 different arrays
    
    One group is peaceful, the other group is non-peaceful
    
    Parameters
    ----------
    data : DataFrame
        DataFrame that contains nan values
        
    Returns:
    -------
    Retures a DataFrame with U_statistic and Pvalue for every column
    
    U_statististic: float
        Mann-Whitney U statistic, equal to min(U for x, and U for y)
        
    p_value: float
        p-value assuming asymptotic normal distribution. Two-Sided.
    """
    data_copy = data.drop('PSys', axis=1)
    res_df = pd.DataFrame(columns=['Variable', 't_statistic', 'p_value'])
    for label in data_copy.columns:
        # isolate the target variable and current variable
        X = data[[label, 'PSys']]
        # drop all Nan values
        X = X.dropna()
        # filter out peaceful rows
        peaceful = X[X.PSys == 1]
        peaceful = peaceful.drop('PSys', axis=1)
        # filter out non peaceful rows
        non_peaceful = X[X.PSys == 2]
        non_peaceful = non_peaceful.drop('PSys', axis=1) 
        # calculates two-sided Mann-Whitney U test
        utest_result = stats.mannwhitneyu(peaceful, non_peaceful, alternative='two-sided')
        ttest_df = pd.DataFrame({'Variable' : [label],
                                 'U_statistic' : utest_result.statistic,
                                 'p_value' : utest_result.pvalue
                                })
        
        res_df = res_df.append(ttest_df)
    
    res_df.set_index('Variable', inplace=True)
    
    return res_df

In [62]:
peace_sys = load_peace_sys_data()

peace_sys.head()

,SCCS,Coder,ID1.1Over,ID1.2Ethno,Int2.1Mar,Int2.2Econ,Int2.3Pol,Int2.4Hist,Dep3.1Sec,Dep3.2Ecol,...,CM8.5Peace,CM8.6War,Lead9.1P,Lead9.2War,Cult10.1Com,Cult10.2Diff,Comp10.3,InComp10.4,PSys,PSysRec
Name,,,,,,,,,,,,,,,,,,,,,
Gilbertese,107,KA,9.0,2.0,3,9.0,9,9,9,9,...,9,9,9,9,9,9,9,9,2,0
Marshallese,108,EC,1.0,9.0,9,1.0,9,9,9,9,...,9,2,9,4,9,9,9,9,2,0
E. Pomo,135,"KA, EC",9.0,9.0,9,9.0,9,9,1,1,...,9,9,2,9,9,9,9,9,2,0
Popoluca,154,KA,9.0,3.0,9,3.0,9,9,9,3,...,9,9,9,9,9,9,9,9,2,0
Konso,35,KA,9.0,2.0,3,4.0,9,4,9,2,...,9,9,9,9,3,9,3,3,2,0


In [63]:
peace_sys = peace_sys.drop(['SCCS','Coder'], axis=1)

In [64]:
peace_sys = peace_sys.replace(9, np.nan)

In [65]:
WAR_VARS = ['WNorm5.2', 'Lead9.2War', 'SymWar6', 'RitWar6', 'WVal4.2', 'PSys']

In [66]:
WAR_VARS

['WNorm5.2', 'Lead9.2War', 'SymWar6', 'RitWar6', 'WVal4.2', 'PSys']

In [67]:
war_dataset = peace_sys[WAR_VARS]

war_dataset

,WNorm5.2,Lead9.2War,SymWar6,RitWar6,WVal4.2,PSys
Name,,,,,,
Gilbertese,NaN,NaN,NaN,NaN,NaN,2
Marshallese,NaN,4.0,2.0,NaN,3.0,2
E. Pomo,2.0,NaN,2.5,4.0,3.0,2
Popoluca,NaN,NaN,NaN,NaN,NaN,2
Konso,4.0,NaN,3.0,NaN,3.0,2
Bribri,3.0,NaN,2.0,2.0,3.0,2
Tallensi,NaN,NaN,NaN,NaN,2.0,2
Russians,NaN,NaN,NaN,NaN,NaN,2
Trukese,3.0,NaN,NaN,NaN,NaN,2


In [68]:
y = war_dataset.PSys
X = war_dataset.drop('PSys', axis=1)

In [69]:
# generating train_set and test_set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.4 , stratify=y, random_state=42)

# Random Forest

In [70]:
# rfc = RandomForestClassifier(n_estimators=1000).fit(X_train, y_train)

# rfc_pred = rfc.predict(X_test)

In [71]:
# print('Accuracy: ', accuracy_score(y_test, rfc_pred))
# print('F1 Score: ', f1_score(y_test, rfc_pred, average='weighted'))
# print('Recall Score: ', recall_score(y_test, rfc_pred, average='weighted'))

In [72]:
# rfc.feature_importances_

# Logistic Regression

In [73]:
logit_results = logit_with_nan(war_dataset)

In [74]:
logit_results

,Logistic_Sig_Level,Logistic_Beta,Logistic_Odds_Ratio
Variable,,,
WNorm5.2,0.005159,1.323790,3.757636
Lead9.2War,0.029607,1.329109,3.777675
SymWar6,0.040487,1.575338,4.832373
RitWar6,0.010238,1.619016,5.048121
WVal4.2,0.003350,1.668282,5.303049


# T-Test

In [75]:
ttest = ttest_with_nan(war_dataset)

In [76]:
ttest

,t_statistic,p_value
Variable,,
WNorm5.2,-3.758143,0.000768
Lead9.2War,-2.900217,0.011637
SymWar6,-2.600997,0.018061
RitWar6,-3.871833,0.000949
WVal4.2,-4.723962,0.000039


# Mann-Whitney U-Test

In [77]:
mannwhitneyu = mannwhitneyu_with_nan(war_dataset)

In [78]:
mannwhitneyu

,t_statistic,p_value
Variable,,
WNorm5.2,43.0,0.001794
Lead9.2War,10.5,0.018286
SymWar6,21.0,0.033359
RitWar6,15.5,0.003076
WVal4.2,58.5,0.000786
